In [3]:
import pandas as pd
import py2neo
import urllib.request as urllib
from py2neo import Graph,Node,Relationship


In [146]:
# esc =  eurovision song contest
esc_1975_2019 =  "eurovision_song_contest_1975_2019.xlsx"
esc_additional = "Eurovision additional.xlsx"
geo_cepii = "geo_cepii.xls"
iso2 = "iso2.csv"

#load data in pandas df
df_iso2 = pd.read_csv("data/" + iso2)
df_esc = pd.read_excel("data/" + esc_1975_2019,sheet_name = "Data",header=0)
df_geo = pd.read_excel("data/" + geo_cepii,header=0)
df_esc_additional = pd.read_excel("data/" + esc_additional, header=0)


In [147]:
## This code converts the "From country" and "To country" columns df_esc to "from_iso2" and "to_iso2"

faulty_countries = []
def get_faulty_countries(country):
    code = df_iso2.loc[df_iso2['Name'] == country]["Code"].values
    if code.size == 0:
        if country not in faulty_countries:
            faulty_countries.append(country)
            
df_esc["From country"].apply(get_faulty_countries)
df_esc["To country"].apply(get_faulty_countries)
            
# manual lookup in iso2file
#in esc files Serbia and montenegro are 1 country TODO
correct_countries = ['Netherlands','Macedonia, the Former Yugoslav Republic of', 'Bosnia and Herzegovina', 'Russian Federation', 'Macedonia, the Former Yugoslav Republic of','Serbia','Moldova, Republic of', 'Macedonia, the Former Yugoslav Republic of','Netherlands', 'Macedonia, the Former Yugoslav Republic of']
df_country_lookup = pd.DataFrame({'faulty': faulty_countries,'correct':correct_countries})

def fix_country(country):
    new_country = df_country_lookup.loc[df_country_lookup["faulty"] == country]["correct"].values 
    if new_country.size > 0:
        return new_country[0]
    else: 
        return Nan
    
def get_iso2(country):
    code = df_iso2.loc[df_iso2['Name'] == country]["Code"].values
    if code.size > 0:
        return code[0]
    else:
        return df_iso2.loc[df_iso2['Name'] == fix_country(country)]["Code"].values[0]

    
df_esc["from_iso2"] = df_esc["From country"].apply(get_iso2)
df_esc["to_iso2"] = df_esc["To country"].apply(get_iso2)

In [148]:
# copy features from df_geo to df_esc based on matching iso2 with "from_iso2" column
geo_features = ['continent', 'langoff_1','lat','lon','colonizer1'] #features that seem interesting? TODO test
for feature in geo_features:
    df_esc[feature] = ""
    for index, row in df_esc.iterrows():
        val = df_geo.loc[df_geo["iso2"]== row['from_iso2']][feature].values
        if val.size > 0:
            df_esc.loc[index,feature] = val[0]
        else:
            row[feature] = "Nan"
    

In [150]:
df_esc

,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Duplicate,from_iso2,to_iso2,continent,langoff_1,lat,lon,colonizer1
0,1975,f,1975f,J,Belgium,Belgium,0,x,BE,BE,Europe,Dutch,50.83333,4.35,.
1,1975,f,1975f,J,Belgium,Finland,0,NaN,BE,FI,Europe,Dutch,50.83333,4.35,.
2,1975,f,1975f,J,Belgium,France,2,NaN,BE,FR,Europe,Dutch,50.83333,4.35,.
3,1975,f,1975f,J,Belgium,Germany,0,NaN,BE,DE,Europe,Dutch,50.83333,4.35,.
4,1975,f,1975f,J,Belgium,Ireland,12,NaN,BE,IE,Europe,Dutch,50.83333,4.35,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49827,2019,f,2019f,J,Belarus,Greece,0,NaN,BY,GR,Europe,Belarussian,53.85,27.5,RUS
49828,2019,f,2019f,J,Belarus,Germany,8,NaN,BY,DE,Europe,Belarussian,53.85,27.5,RUS
49829,2019,f,2019f,J,Belarus,Belarus,0,x,BY,BY,Europe,Belarussian,53.85,27.5,RUS
49830,2019,f,2019f,J,Belarus,The Netherlands,0,NaN,BY,NL,Europe,Belarussian,53.85,27.5,RUS


In [6]:
# Connect to neo4j database
f=open("authentication/neo4j.txt","r")
lines=f.readlines()
user=lines[0].replace('\n','')
pwd=lines[1].replace('\n','')
http_port = lines[2]
f.close()

graph = Graph(http_port, auth=(user, pwd))

In [7]:
# laod data in neo4j database
# //Eurovision votes in the 2012,2013, 2014 and 2015 final session.
fp = urllib.urlopen("https://goz39a.s3.eu-central-1.amazonaws.com/eurovision/eurovision.txt")
mybytes = fp.read()
data = mybytes.decode("utf8")
fp.close()
graph.run(data)
graph.run("MATCH (n) RETURN count(*)")

count(*)
46


In [6]:

def clear_graph(): 
    return (graph.run("MATCH (n)-[r]->() DELETE r"), graph.run("MATCH (n) DELETE n"))
    
clear_graph()

((No data), (No data))